Importo las librerías correspondientes: 

In [ ]:
import os
import csv
from googletrans import Translator
from httpcore import TimeoutException

Establezco la ruta de los archivos:


In [ ]:
def file_path(one_string):
    '''ruta de acceso al archivo'''
    base_path = os.path.abspath(os.path.dirname("__file__"))
    archives= os.path.join(base_path, one_string)
    return archives

volcano_archives = "volcanic_input.csv"
volcano_spanish = "dataset_volcanic.csv"

file_path(volcano_archives)
file_path(volcano_spanish)


Funciones del programa:


In [ ]:
def translator_volcano_type(data):
  '''Traduce el string pasado por parámetro al español'''
  translator = Translator()
  return translator.translate(data, dest='es').text.title()
  
def booleans_flags(data, one_string,):
  '''devuelve una lista con True en los lugares donde se encuentra el string pasado por parámetro,
   sino devuelve '¿?'''
  return True if data== one_string else '¿?'

def unifier_name_country_columns (data_name, data_country):
  '''devuelve el nombre y el país, entre paréntesis, de cada volcán'''
  return "{name} ({country})".format(name = data_name, country = data_country)

def data_volcanic_eruption(retries=0):
  if retries < 1: 
    try: 
     new_data = (line[0], line[10], translator_volcano_type(line[9]), booleans_flags(
     line[3], 'Tsunami'), booleans_flags(line[4], 'Earthquake'), unifier_name_country_columns (line[5], line[7]))
     return writer.writerow(new_data)
    except TimeoutException:
     print('Tenemos una demora con la comunicación con Google, seguimos intentando') 
     retries+=1 
     data_volcanic_eruption(retries)


Programa:

In [ ]:
try:
    with open(volcano_archives, encoding = "utf8") as data_set:
        reader = csv.reader(data_set, delimiter = ";")
        header, data_file_input = next(reader), list(reader)

    # sobre escribo el header
        header = ['Year', 'Volcanic Explosivity Index', 'Volcano Type', 'Flag Tsunami', 'Flag Earthquake', 'Name and country']
    # creo un archivo .csv de new_file
    with open(volcano_spanish, "w", encoding = "utf8",  newline = "") as new_file:
        writer = csv.writer(new_file)
       # volcano_archives = next() #salteo el encabezado del archivo
        writer.writerow(header)  # agrego el nuevo encabezado
        for line in data_file_input:
         data_volcanic_eruption()       
except FileNotFoundError:
    print('ERROR: No se ha encontrado el archivo.')